# WMA Charting Notebook

#### Packages and Tools

In [1]:
from utils.packages import *
from utils.tools import *
%matplotlib inline

#### Read from config file

In [2]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)
    
temp_file = 'config/wma.yaml'
with open(temp_file) as temp_file:
    wma = yaml.load(temp_file)    

In [3]:
instrument = wma['parameter']['instrument']
short_moving_window = wma['parameter']['short_moving_window']
long_moving_window = wma['parameter']['long_moving_window']
lists_size = wma['parameter']['lists_size']

access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']

#### Code Engine

In [ ]:
api = API(access_token = access_token)
params = {'instruments': instrument}
order_flag = 'not_ordered'
r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)

short_wma = 0
short_flag = True
short_list = collections.deque([])
short_wma_list = collections.deque([])
short_weights = get_weights(short_moving_window)

long_wma = 0
long_flag = True
long_list = collections.deque([])
long_wma_list = collections.deque([])
long_weights = get_weights(long_moving_window)

tick_list = collections.deque([])

start_time = time.time()
time_flag = True 
long_list_counter = 0

list_ready = False

for i, resp in enumerate(rv):        
    resp_type = resp['type']       

    # Heart beat response to keep the api connection alive (Avoid timeout)
    if resp_type == 'HEARTBEAT': 
        print('1')
        pass
    #-----------------------------------------------------------------------------------------------------
    
    
    # Price response 
    elif resp_type == 'PRICE' and order_flag == 'not_ordered': # Check if we are yet to make the order               
        
        #Decode values from response
        date_val, time_val, time_fraction = get_date_time(resp) # Get time stamp for reference            
        sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response 
        print('2')
        #-----------------------------------------------------------------------------------------
        
        #Prepare short list to calculate WMA of shortwindows
        if len(short_list) < short_moving_window:
            short_list.append(tick_price)
            print('3')

        # Calculate wma for first iteration
        if short_flag and len(short_list) == short_moving_window:
            short_wma = sum(np.array(short_list)*short_weights)
            short_flag =  False
            print('4')

        # Calculate wma from the second iteration
        if short_flag == False and len(short_list) == short_moving_window:
            short_list.popleft()   
            short_list.append(tick_price) 
            short_wma = sum(np.array(short_list)*short_weights)
            print('5')

        # Append WMA list
        if len(short_list) == short_moving_window:
            print('6')
            if len(short_wma_list) < lists_size:
                short_wma_list.append(short_wma)
                print('7')

            elif len(short_wma_list) == lists_size:
                short_wma_list.popleft()   
                short_wma_list.append(short_wma) 
                print('8')
                
        #print(f'short_wma_list : {len(short_wma_list)}')
        #-----------------------------------------------------------------------------------------
        
        
        #Prepare long list to calculate WMA of longwindows
        if len(long_list) < long_moving_window:
            long_list.append(tick_price)
            print('9')
            
         # Calculate wma for first iteration
        if long_flag and len(long_list) == long_moving_window:
            long_wma = sum(np.array(long_list)*long_weights)
            long_flag =  False
            print('10')

        # Calculate wma from the second iteration
        if long_flag == False and len(long_list) == long_moving_window:
            long_list.popleft()   
            long_list.append(tick_price) 
            long_wma = sum(np.array(long_list)*long_weights)
            print('11')

        # Append WMA list
        if len(long_list) == long_moving_window:
            print('12')
            if len(long_wma_list) < lists_size:
                long_wma_list.append(long_wma)
                tick_list.append(tick_price)
                print('13')

            # If WMA list reached required size start plotting
            elif len(long_wma_list) == lists_size:
                print('14')
                if time_flag:
                    end_time = time.time()    
                    duration = calc_duration(start_time, end_time)
                    time_flag = False
                    print('15')
                plt.close()
                print('16')
                long_wma_list.popleft()   
                long_wma_list.append(long_wma)  
                
                tick_list.popleft()   
                tick_list.append(tick_price)
                
                plt.figure(figsize=(20,5))
                plt.plot(short_wma_list, 'r' ,label="short wma")
                plt.plot(long_wma_list,'b' ,label="long wma")
                plt.plot(tick_list, 'y' ,label="tick")
                plt.legend(loc="upper left")
                display.display(plt.gcf())
                display.clear_output(wait = True)                
                print('17')